In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load fine-tuned model
model = AutoModelForSequenceClassification.from_pretrained("bucketresearch/politicalBiasBERT")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# # Set model to eval mode and use GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# model.eval()


In [18]:
text = "President Trump is the best America has ever had"

# Tokenize
inputs = tokenizer(text, return_tensors="pt")
labels = torch.tensor([0])
outputs = model(**inputs, labels=labels)
loss, logits = outputs[:2]
print(logits.softmax(dim=-1)[0].tolist()) # Result in [Left, Center, Right]

[0.18539030849933624, 0.19757050275802612, 0.617039144039154]


In [ ]:
# The Issue is that it's trained to recognize political bias in news. It needs fine-tuning on small texts